# Statistical visualization with Seaborn

In the previous notebook, you saw how to operate on Matplotlib objects. This serves as an important foundation. But now, I am going to show you a short-cut that will allow you to create elegant and rather complex visualizations of statistical quantities, without having to operate at that low level. This is the Seaborn visualization library. It builds upon the Matplotlib basis to 